# Data Fetching and Preprocessing

Code in this notebook is used for downloading and processing data (from the CDC and other sources) to prepare for analysis. The data from this step is placed in the `data/` folder.

**For Air Quality Data:**

Since we are trying see the effects of ozone and PM 2.5 concentration on chronic diseases, we are worried more about extended air quality patterns, rather than specific years of data. It is much more likely that chronic diseases will stem from extended exposure to air pollutants, rather than a small time period. We take the average over all the data available (January 2011 to December 2014), which assumes that this period is at least relatively representative of the long-term patterns before 2011.

**For Population Data:**

We load data from the US Census website and consolidate the population estimates into subgroups.

**For CDC Chronic Disease Indicators:**

Downloads and stores CDI dataset in data folder.

**For Other Data:**

Some preprocessing steps (for Diabetes Prevalence by Age/Sex) were not included in this notebook for brevity. The preprocessed file result is included in the data folder, and code to produce the data is also included.

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from sodapy import Socrata
from scripts.aqi_state_agg import get_cdc_data, agg_county_weighted_mean

In [3]:
PATH_TO_DATA = "../data"

## Ozone Data Aggregation

Downloads data from the CDC's [Daily Census Tract-Level Ozone Concentrations](https://data.cdc.gov/Environmental-Health-Toxicology/Daily-Census-Tract-Level-Ozone-Concentrations-2011/372p-dx3h) and aggregates county-level data to state-level using a weighted mean based on county's population. We do this for the first day of every month from January 2011 to December 2014, then take the mean over all of these to calculate the average ozone concentration for the 4 years.

The resulting DataFrame is saved as a csv file at `data/average_ozone_per_state.csv`

In [4]:
months = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]
dates = sum([["01"+m+str(yr) for m in months] for yr in np.arange(2011, 2015)], [])
ozone = get_cdc_data(dates, ozone=True)

100%|██████████| 48/48 [00:23<00:00,  2.03it/s]


In [5]:
ozone_by_state = agg_county_weighted_mean(ozone, data_dir=PATH_TO_DATA)
ozone_by_state = ozone_by_state.set_index('abbrev')

In [6]:
ozone_by_state['average o3'] = ozone_by_state.drop(columns='state').mean(axis=1)
ozone_by_state['median o3'] = ozone_by_state.drop(columns=['state', 'average o3']).median(axis=1)
ozone_by_state = ozone_by_state[['state', 'average o3', 'median o3']]
ozone_by_state.to_csv(PATH_TO_DATA+"/average_ozone_by_state.csv")

## PM 2.5 Data Aggregation

Downloads data from the CDC's [Daily Census Tract-Level PM2.5 Concentrations](https://data.cdc.gov/Environmental-Health-Toxicology/Daily-Census-Tract-Level-PM2-5-Concentrations-2011/fcqm-xrf4) and aggregates county-level data to state-level using a weighted mean based on county's population. We do this for the first day of every month from January 2011 to December 2014, then take the mean over all of these to calculate the average ozone concentration for the 4 years.

The resulting DataFrame is saved as a csv file at `data/average_pm_by_state.csv`

In [7]:
months = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]
dates = sum([["01"+m+str(yr) for m in months] for yr in np.arange(2011, 2015)], [])
pm25 = get_cdc_data(dates, ozone=False)

100%|██████████| 48/48 [00:22<00:00,  2.14it/s]


In [8]:
pm25_by_state = agg_county_weighted_mean(pm25, data_dir=PATH_TO_DATA)
pm25_by_state = pm25_by_state.set_index('abbrev')

In [9]:
yearly = {}
for i in range(4):
    yearly[f'{2011+i}'] = pm25_by_state.iloc[:, (i*12):((i+1)*12)].mean(axis=1)
yearly_pm25 = pd.DataFrame(yearly)
yearly_pm25.to_csv(PATH_TO_DATA + "/yearly_pm_by_state.csv")
yearly_pm25.head()

,2011,2012,2013,2014
abbrev,,,,
AL,11.748242,11.454434,9.714757,12.506239
AZ,7.838444,9.232254,7.924441,8.970285
AR,11.059018,9.843660,9.833751,10.626165
CA,9.746000,10.029275,11.017674,11.995410
CO,5.870499,6.268958,5.951824,7.624095


In [10]:
pm25_by_state['average pm'] = pm25_by_state.drop(columns='state').mean(axis=1)
pm25_by_state['median pm'] = pm25_by_state.drop(columns=['state', 'average pm']).median(axis=1)
pm25_by_state = pm25_by_state[['state', 'average pm', 'median pm']]
pm25_by_state.to_csv(PATH_TO_DATA + "/average_pm_by_state.csv")

## Population by Age/Sex

The CSV data is downloaded from https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/state/asrh/SC-EST2020-AGESEX-CIV.csv.

Details about the table format can be found here: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2020/sc-est2020-18+pop-res.pdf

In [11]:
pop_agesex_path = PATH_TO_DATA + "/state_pop_by_agesex.csv"

In [12]:
!curl -o {pop_agesex_path} https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/state/asrh/SC-EST2020-AGESEX-CIV.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1297k    0 1297k    0     0  2060k      0 --:--:-- --:--:-- --:--:-- 2062k


In [13]:
data = pd.read_csv(pop_agesex_path)
pop2014 = data[(data['SUMLEV'] == 40) & (data['AGE'] < 100) & (data["SEX"] > 0)][['NAME', 'SEX', 'AGE', 'POPEST2014_CIV']]

In [14]:
# State name to 2-letter Postal Code
abbrevs = pd.read_csv(PATH_TO_DATA + "/county_to_state_aggregation/state_abbreviations.csv")

### By Sex, 2014

In [15]:
fn = lambda state_df: state_df.groupby("SEX").agg({'POPEST2014_CIV': np.sum})
gender = pop2014.groupby(["NAME"]).apply(fn)

gender = gender.reset_index()
gender = gender.pivot(index="NAME", columns="SEX", values="POPEST2014_CIV")

gender.index = gender.index.rename("State")
gender = gender.rename(columns={1:'male', 2:'female'})

gender.to_csv(PATH_TO_DATA + "/state_pop_by_gender_2014.csv")

### By Age Group, 2014

In [16]:
age_groups = [18, 45, 65, 75, 100]
fn = lambda state_df: state_df.groupby(pd.cut(state_df['AGE'], bins=age_groups, right=False)).agg({"POPEST2014_CIV": np.sum})
age = pop2014.groupby(["NAME"]).apply(fn)

age = age.reset_index()
age = age.pivot(index="NAME", columns='AGE', values="POPEST2014_CIV")

age.index = age.index.rename("State")
age.columns = ["18-44", "45-64", "65-74", "75+"]

age.to_csv(PATH_TO_DATA + "/state_pop_by_age_group_2014.csv")

### High school population size, 2013

Here, we simplify "high school student" to be anyone ages 15 to 18 (inclusive). This will likely overestimate the true number, but we believe the relative counts between states will still be relatively close to the true values.

In [17]:
pop2013 = pd.read_csv(pop_agesex_path)
pop2013 = pop2013[(pop2013['SUMLEV'] == 40) & (pop2013['AGE'] < 100) & (pop2013["SEX"] > 0)][['NAME', 'SEX', 'AGE', 'POPEST2013_CIV']]

hs = pop2013[pop2013['AGE'].between(14, 18)].groupby("NAME").agg({'POPEST2013_CIV': np.sum})
hs = hs.merge(abbrevs, left_index=True, right_on='State')
hs = hs.rename(columns={"POPEST2013_CIV": "num HS age"})
hs.to_csv(PATH_TO_DATA + "/hs_aged_pop_2013.csv")

### Adults (18 or older), 2014

In [18]:
adult = pop2014[pop2014['AGE'].between(18, 100)].groupby("NAME").agg({'POPEST2014_CIV': np.sum})
adult = adult.merge(abbrevs, left_index=True, right_on='State')
adult = adult.rename(columns={"POPEST2014_CIV": "adult population"})
adult.to_csv(PATH_TO_DATA + "/adult_pop_2014.csv")

## Download CDI Data

In [19]:
# downloads CDI dataset and save as cdi.csv
!curl -o {PATH_TO_DATA + "/cdi.csv"} "https://chronicdata.cdc.gov/api/views/g4ie-h725/rows.csv?accessType=DOWNLOAD"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  342M    0  342M    0     0  8455k      0 --:--:--  0:00:41 --:--:-- 8832k


## Diabetes Prevalence by Age/Sex

In [21]:
categorical_diabetes = pd.read_csv(PATH_TO_DATA + '/categorical_diabetes.csv')
categorical_diabetes.head()

,State,male,female,18-44,45-64,65-74,75+
0,Alabama,12.0,11.5,4.8,17.8,24.3,21.0
1,Alaska,6.7,8.6,1.8,10.3,21.3,21.0
2,Arizona,10.1,8.3,2.8,14.0,21.8,18.7
3,Arkansas,11.9,11.3,3.9,17.9,24.7,22.8
4,California,10.9,9.0,2.9,15.4,21.5,22.5
